## Задача 3.2
[**Выборка:**](https://github.com/natasha/nerus) набор предложений на русском языке с указанием именованных сущностей (они же NE, named entity). 

Требуется:
1. Рассмотреть последовательность именованных и неименованных сущностей как марковскую модель. Определить оптимальный порядок марковской модели.
2. Обучить скрытую марковскую модель по выборке. Оценить точность предсказания, является ли слово именованной сущностью или нет, посчитать энтропию на выборке.

*Важно:* в целях ускорения эксперимента рекомендуется взять первые 10 МБ текста из выборки.

In [4]:
from re import compile, Pattern
from sys import getsizeof
from nerus import load_nerus
from nltk.tag import hmm
import nltk
from collections import Counter
from more_itertools import take
from scipy.stats.distributions import chi2
from pandas import DataFrame
from seaborn import heatmap
from matplotlib.pyplot import figure
from numpy import transpose, log, sum as nsum, unique as nunique

In [8]:
nerus_dataset = load_nerus('data/nerus_lenta.conllu.gz')

In [9]:
def words_getter(rus_re, mb_count, dataset):
    """gets word and its pos for every word in dataset"""
    tokens = []
    megabyte_size = 1024 ** 2
    i = 0
    while getsizeof(tokens) < mb_count * megabyte_size:
        i += 1
        for sent in next(dataset).sents:
            for token in sent.tokens:
                word = token.text.lower()
                pos = token.pos
                if russian.match(word):
                    tokens.append((word, pos))
    return tokens

russian = compile('^[а-я]+$')

tokens = words_getter(russian, 10, nerus_dataset)
print(f"Number of tokens: {len(tokens)}")

Number of tokens: 1188214


In [10]:
tokens[:10]

[('по', 'ADP'),
 ('социальным', 'ADJ'),
 ('вопросам', 'NOUN'),
 ('татьяна', 'PROPN'),
 ('голикова', 'PROPN'),
 ('рассказала', 'VERB'),
 ('в', 'ADP'),
 ('каких', 'DET'),
 ('регионах', 'NOUN'),
 ('россии', 'PROPN')]

Получим текст.

In [12]:
text = ' '.join(transpose(tokens)[0])
len(text), text[:100]

(8540838,
 'по социальным вопросам татьяна голикова рассказала в каких регионах россии зафиксирована наиболее вы')

Сделаем словарь, в котором ключи это слова, а значения - это части речи.

In [13]:
vocab = {word: pos for word, pos in tokens}
len(vocab)

98654

Посчитаем n-grams

In [14]:
def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

In [15]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(tokens, 3)

In [16]:
def unigram_probas(ngram_counts):
    """
    gets probabilities of unigrams in text
    """
    p1 = {}
    n = sum(ngram_counts[1].values())
    for w in ngram_counts[1]:
        p1[w] = ngram_counts[1][w] / n
    return p1


def bigram_probas(ngram_counts, prefix_counts):
    """
    gets probabilities of bigrams in text
    """
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2


def trigram_probas(ngram_counts, prefix_counts):
    """
    gets probabilities of trigrams in text
    """
    p3 = {}
    for w in ngram_counts[3]:
        pre_w = w[:2] + tuple(['*'])
        p3[u'{2}|{1},{0}'.format(*w)] = ngram_counts[3][w] / prefix_counts[3][pre_w]
    return p3

Посчитаем вероятности для каждой из n-грамм для n=1,2,3. И посмотрим на случайные n-граммы и их вероятности.

In [17]:
def random_n_dict_items(d: dict, n: int) -> dict:
    """gets random n items of a sorted dict by values"""
    return [(d[x], x) for x in take(n, d)]

In [18]:
p1 = unigram_probas(ngram_counts)
random_n_dict_items(p1, 10)

[(0.010653804786006561, (('по', 'ADP'),)),
 (1.1782389367571835e-05, (('социальным', 'ADJ'),)),
 (0.00011024950051085074, (('вопросам', 'NOUN'),)),
 (5.049595443245072e-05, (('татьяна', 'PROPN'),)),
 (5.049595443245072e-06, (('голикова', 'PROPN'),)),
 (0.0003686204673568903, (('рассказала', 'VERB'),)),
 (0.05012985876281545, (('в', 'ADP'),)),
 (3.366396962163381e-05, (('каких', 'DET'),)),
 (0.00013718067620815778, (('регионах', 'NOUN'),)),
 (0.003411001721912046, (('россии', 'PROPN'),))]

In [19]:
p2 = bigram_probas(ngram_counts, prefix_counts)
random_n_dict_items(p2, 10)

[(0.0004739710877636464, "('социальным', 'ADJ')|('по', 'ADP')"),
 (0.14285714285714285, "('вопросам', 'NOUN')|('социальным', 'ADJ')"),
 (0.015267175572519083, "('татьяна', 'PROPN')|('вопросам', 'NOUN')"),
 (0.06666666666666667, "('голикова', 'PROPN')|('татьяна', 'PROPN')"),
 (0.16666666666666666, "('рассказала', 'VERB')|('голикова', 'PROPN')"),
 (0.0776255707762557, "('в', 'ADP')|('рассказала', 'VERB')"),
 (0.0002014605892722236, "('каких', 'DET')|('в', 'ADP')"),
 (0.075, "('регионах', 'NOUN')|('каких', 'DET')"),
 (0.15337423312883436, "('россии', 'PROPN')|('регионах', 'NOUN')"),
 (0.0002467308166790032, "('зафиксирована', 'VERB')|('россии', 'PROPN')")]

In [20]:
p3 = trigram_probas(ngram_counts, prefix_counts)
random_n_dict_items(p3, 10)

[(0.3333333333333333,
  "('вопросам', 'NOUN')|('социальным', 'ADJ'),('по', 'ADP')"),
 (1.0, "('татьяна', 'PROPN')|('вопросам', 'NOUN'),('социальным', 'ADJ')"),
 (1.0, "('голикова', 'PROPN')|('татьяна', 'PROPN'),('вопросам', 'NOUN')"),
 (0.25, "('рассказала', 'VERB')|('голикова', 'PROPN'),('татьяна', 'PROPN')"),
 (1.0, "('в', 'ADP')|('рассказала', 'VERB'),('голикова', 'PROPN')"),
 (0.029411764705882353,
  "('каких', 'DET')|('в', 'ADP'),('рассказала', 'VERB')"),
 (0.16666666666666666, "('регионах', 'NOUN')|('каких', 'DET'),('в', 'ADP')"),
 (0.6666666666666666,
  "('россии', 'PROPN')|('регионах', 'NOUN'),('каких', 'DET')"),
 (0.04, "('зафиксирована', 'VERB')|('россии', 'PROPN'),('регионах', 'NOUN')"),
 (1.0, "('наиболее', 'ADV')|('зафиксирована', 'VERB'),('россии', 'PROPN')")]

In [21]:
def chi2_statistic(p_small, p_big, tokens, model_type: str):
    """
    gets chi2 stat for the hypothesis that we can reduce
    the bigram model to the monogram one
    """
    stat_small = []
    stat_big = []
    n = len(tokens)
    for i in range(n - 2):
        if model_type == "3_to_2":
            w = tokens[i : i + 3]
            ngram_big = '{2}|{1},{0}'.format(*w)
            ngram_small = '{1}|{0}'.format(*w)
        elif model_type == "2_to_1":
            w = tokens[i : i + 2]
            ngram_big = '{1}|{0}'.format(*w)
            ngram_small = '{0}'.format(*w)
        else:
            raise ValueError("unrecognized model type")

        try:
            stat_small.append(log(p_small[ngram_small]))
        except KeyError:
            stat_small.append(log(1e-20))
        try:
            stat_big.append(log(p_big[ngram_big]))
        except KeyError:
            stat_big.append(log(1e-20))
    return - 2 * nsum(stat_small) + 2 * nsum(stat_big)

Посмотрим, можем ли мы использовать би-граммы вместо три-грамм

In [22]:
stat_32 = chi2_statistic(p2, p3, tokens, model_type="3_to_2")
print(f'p-value of a chi2 test that we can reduce tri-gram to bi-gram model: {1 - chi2(len(p3) * ((len(p3) - 1) ** 2) - 1).cdf(stat_32):.2f}')

p-value of a chi2 test that we can reduce tri-gram to bi-gram model: 1.00


Посмотрим, можем ли мы использовать монограммы вместо три-грамм

In [24]:
stat_21 = chi2_statistic(p1, p2, tokens, model_type="2_to_1")
print(f'p-value of a chi2 test that we can reduce bi-gram to mono-gram model: {1 - chi2(len(p2) * ((len(p2) - 1) ** 2) - 1).cdf(stat_21):.2f}')

p-value of a chi2 test that we can reduce bi-gram to mono-gram model: 1.00


оптимальная модель пользуется только монограммами

In [25]:
train_data = words_getter(russian, 10, nerus_dataset)

In [26]:
trainer = hmm.HiddenMarkovModelTrainer([])
tagger = trainer.train_supervised([train_data])

In [27]:
train_res = tagger.test([train_data[:len(train_data) // 200]], verbose=True)

Test: житель/NOUN австралии/PROPN убил/VERB питона/NOUN длиной/NOUN более/ADV четырех/NUM метров/NOUN в/ADP попытке/NOUN спасти/VERB своего/DET внука/NOUN об/ADP этом/PRON сообщает/VERB историю/NOUN о/ADP спасении/NOUN рассказала/VERB мать/NOUN пострадавшего/VERB мальчика/NOUN аманда/PROPN ратленд/PROPN во/ADP время/NOUN нападения/NOUN наиш/PROPN играл/VERB вместе/ADV с/ADP трехлетней/ADJ сестрой/NOUN на/ADP веранде/NOUN дома/NOUN в/ADP северном/ADJ квинсленде/PROPN ратленд/PROPN посмотрела/VERB на/ADP потерянное/ADJ лицо/NOUN дочери/NOUN и/CCONJ поняла/VERB что/SCONJ с/ADP детьми/NOUN происходит/VERB неладное/ADJ выйдя/VERB на/ADP улицу/NOUN она/PRON увидела/VERB что/SCONJ питон/NOUN длиной/NOUN метра/NOUN обвился/VERB вокруг/ADP ее/DET маленького/ADJ сына/NOUN змея/NOUN также/ADV начала/VERB кусать/VERB его/PRON за/ADP правую/ADJ руку/NOUN женщина/NOUN не/PART смогла/VERB самостоятельно/ADV избавиться/VERB от/ADP рептилии/NOUN и/CCONJ позвала/VERB на/ADP помощь/NOUN своего/DET отца/N

In [28]:
test_data = words_getter(russian, 0.5, nerus_dataset)

In [29]:
test_res = tagger.test([test_data], verbose=True)

Test: австралийский/ADJ бренд/NOUN выпустил/VERB кружевное/ADJ нижнее/ADJ белье/NOUN для/ADP мужчин/NOUN об/ADP этом/PRON сообщает/VERB коллекция/NOUN включает/VERB себя/PRON множество/NOUN моделей/NOUN бюстгальтеров/NOUN и/CCONJ трусов/NOUN изготовленных/VERB из/ADP шелка/NOUN сатина/NOUN и/CCONJ кружева/NOUN все/DET модели/NOUN представлены/VERB в/ADP пяти/NUM размерах/NOUN от/ADP до/ADP моя/DET девушка/NOUN в/ADP восторге/NOUN когда/SCONJ я/PRON надеваю/VERB бюстгальтер/NOUN и/CCONJ трусы/NOUN в/ADP тон/NOUN написал/VERB довольный/ADJ клиент/NOUN в/ADP отзывах/NOUN к/ADP одному/NUM из/ADP топов/NOUN они/PRON потрясающие/ADJ очень/ADV качественные/ADJ и/CCONJ сексуальные/ADJ одобрил/VERB другой/ADJ покупатель/NOUN белье/NOUN продается/VERB на/ADP официальном/ADJ сайте/NOUN бренда/NOUN по/ADP цене/NOUN от/ADP до/ADP долларов/NOUN в/ADP августе/NOUN французская/ADJ компания/NOUN представила/VERB свою/DET первую/ADJ коллекцию/NOUN макияжа/NOUN для/ADP мужчин/NOUN в/ADP линию/NOUN вошли/

Всё работает довольно плохо, возможно, недостаточный объём обучающей выборки для модели.